In [2]:
from pyfiles import (parametrics as par,
    complex as cplx,
    matrix as mat,
    vineyard as vin,
    input as inp,
    plot as ourplot,
    grid as grid,
    utils as utils,
    examples as ex,
    sneaky_matrix as sneaky
)
from pyfiles.examples import CameraOpt

# so we don't have to reboot kernel every time we modify classes.py
%load_ext autoreload
%autoreload 2
%env RUST_BACKTRACE=1

import numpy as np
from copy import deepcopy

from typing import *

import matplotlib.pyplot as plt
%matplotlib inline

import mars

filename = 'input/cube.obj'

class vineyard2(vin.vineyard):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.swaps = []

  def get_point_key(self, point: np.ndarray):
    return None
  
  def prune(self, s1: cplx.simplex, s2: cplx.simplex, old_state, new_state):
    if s1.dim() == s2.dim():
      self.swaps.append((s1, s2, old_state, new_state))
    return True

p0 = [0.0, 0.0, 0.0]
p1 = [0.1, 0.0, 0.0]


py_complex = inp.read_obj(filename)
py_vineyard = vineyard2(py_complex)
py_state0 = py_vineyard.reduce(p0)
py_state1 = py_vineyard.reduce_vine(py_state0, p1)

print([(a, b) for (a, b, _, _) in py_vineyard.swaps])


rs_complex = mars.read_from_obj(filename)
rs_state0 = mars.reduce_from_scratch(rs_complex, p0)
swaps = mars.vineyards_123(rs_complex, rs_state0, p1)
print(swaps)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: RUST_BACKTRACE=1
[(simplex e8 bd [6, 7], simplex e7 bd [4, 6]), (simplex v2 bd [-1], simplex v0 bd [-1])]
[(0, (2, 5)), (1, (2, 14))]


In [8]:
print(rs_complex.simplices_per_dim[0][2])
print(rs_complex.simplices_per_dim[0][5])

print()
print(py_complex.vertlist[0], py_complex.vertlist[0].coords)
print(py_complex.vertlist[2], py_complex.vertlist[2].coords)

v(2; pos=[1, 1, -1]
v(5; pos=[-1, -1, 1]

simplex v0 bd [-1] [-1.  1. -1.]
simplex v2 bd [-1] [ 1.  1. -1.]
